# Construction d'un dataset à features numériques intéressantes

In [1]:
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np

In [2]:
d = pd.read_pickle("../Création du Dataset/datasetv3.pkl")
dataset = d.copy()

# Fonctions de conversion 

In [3]:
def min_max_moy(datasetOutput,datasetInterm,colname,teamnum):
    datasetOutput[colname+teamnum+"_MIN"] = datasetInterm[['TOP'+teamnum+'_'+colname,'JGL'+teamnum+'_'+colname,'MID'+teamnum+'_'+colname,'ADC'+teamnum+'_'+colname,'SUP'+teamnum+'_'+colname]].min(axis=1)
    datasetOutput[colname+teamnum+"_MAX"] = datasetInterm[['TOP'+teamnum+'_'+colname,'JGL'+teamnum+'_'+colname,'MID'+teamnum+'_'+colname,'ADC'+teamnum+'_'+colname,'SUP'+teamnum+'_'+colname]].max(axis=1)
    datasetOutput[colname+teamnum+"_MOY"] = datasetInterm[['TOP'+teamnum+'_'+colname,'JGL'+teamnum+'_'+colname,'MID'+teamnum+'_'+colname,'ADC'+teamnum+'_'+colname,'SUP'+teamnum+'_'+colname]].mean(axis=1)

### Conversion du Rank
Convertir le rang en nombre de LP au dessus de Iron IV 0 LP

In [22]:
def add_rank_numerical_value_columns(datasetInput,datasetOutput):
    listRankCols = ['TOP0_RANK','TOP1_RANK','JGL0_RANK','JGL1_RANK','MID0_RANK','MID1_RANK','ADC0_RANK','ADC1_RANK','SUP0_RANK','SUP1_RANK']
    rankOrder = ['IRON','BRONZE','SILVER','GOLD','PLATINUM','DIAMOND','MASTER','GRANDMASTER','CHALLENGER']
    divisionOrder = ['IV','III','II','I']
    datasetInterm = pd.DataFrame()
    
    for col in listRankCols:
        rank = pd.DataFrame(datasetInput[col].tolist(),datasetInput.index,columns=['RANK_','DIVISION','LP'])
        for i in range(len(rankOrder)):
            rank.loc[rank['RANK_']==rankOrder[i], 'RANK_'] = i

        for i in range(len(divisionOrder)):
            rank.loc[rank['DIVISION']==divisionOrder[i], 'DIVISION'] = i
        
        datasetInterm[col+'_'] = rank['RANK_']*400 + rank['DIVISION']*100 + rank['LP']
    
    min_max_moy(datasetOutput,datasetInterm,'RANK_','0')
    min_max_moy(datasetOutput,datasetInterm,'RANK_','1')
    

In [52]:
#Test
df = pd.DataFrame()
add_rank_numerical_value_columns(dataset,df)
df

,RANK_0_MIN,RANK_0_MAX,RANK_0_MOY,RANK_1_MIN,RANK_1_MAX,RANK_1_MOY
EUW1_6140228604,1275.0,1800.0,1543.2,1360.0,1620.0,1550.0
EUW1_6141546191,1271.0,1863.0,1708.8,1822.0,1946.0,1851.6
EUW1_6140416681,825.0,1600.0,1211.8,1122.0,1744.0,1380.2
EUW1_6139592219,1530.0,1751.0,1640.4,1663.0,1816.0,1754.4
EUW1_6045620377,1260.0,2030.0,1701.2,1600.0,1939.0,1731.8
...,...,...,...,...,...,...
EUW1_6135052656,1465.0,1674.0,1606.4,1390.0,1700.0,1586.4
EUW1_6141595905,1276.0,1738.0,1571.8,1238.0,2000.0,1502.4
EUW1_6137361607,694.0,1720.0,1170.2,452.0,1558.0,1052.6
EUW1_6088300233,1001.0,2231.0,1561.4,665.0,1714.0,1317.0


### Conversion des KDA/KDAG
Calcul du ratio (K+A)/D

In [18]:
def add_kda_ratio_columns(datasetInput,datasetOutput):
    listRankCols = ['TOP0_KDA','TOP1_KDA','JGL0_KDA','JGL1_KDA','MID0_KDA','MID1_KDA','ADC0_KDA','ADC1_KDA','SUP0_KDA','SUP1_KDA']
    datasetInterm = pd.DataFrame()
    
    for col in listRankCols:
        kda = pd.DataFrame(datasetInput[col].tolist(),datasetInput.index,columns=['K','D','A'])
        datasetInterm[col+'_log_'] = (kda['K'] + kda['A']) / (kda['D']+1)
        kdag = pd.DataFrame(datasetInput[col+'G'].tolist(),datasetInput.index,columns=['K','D','A'])
        datasetInterm[col+'G'+'_log_'] = (kdag['K'] + kdag['A']) / (kdag['D']+1)
    
    datasetInterm.applymap(lambda x:np.log(1+x))
    
    min_max_moy(datasetOutput,datasetInterm,'KDA_log_','0')
    min_max_moy(datasetOutput,datasetInterm,'KDA_log_','1')
    min_max_moy(datasetOutput,datasetInterm,'KDAG_log_','0')
    min_max_moy(datasetOutput,datasetInterm,'KDAG_log_','1')
    

In [19]:
#Test
df = pd.DataFrame()
add_kda_ratio_columns(dataset,df)
df

,KDA_log_0_MIN,KDA_log_0_MAX,KDA_log_0_MOY,KDA_log_1_MIN,KDA_log_1_MAX,KDA_log_1_MOY,KDAG_log_0_MIN,KDAG_log_0_MAX,KDAG_log_0_MOY,KDAG_log_1_MIN,KDAG_log_1_MAX,KDAG_log_1_MOY
EUW1_6140228604,0.0,2.750000,0.956061,0.0,4.642857,1.991005,1.456522,2.000000,1.743835,1.368421,3.700000,2.726277
EUW1_6141546191,0.0,5.666667,2.354762,0.0,2.545455,1.516498,1.552632,3.250000,2.407814,1.375000,2.333333,1.833788
EUW1_6140416681,0.0,0.000000,0.000000,0.0,1.600000,0.473846,1.631579,3.450000,2.465020,1.311111,2.724138,2.032696
EUW1_6139592219,0.0,1.275862,0.486751,1.0,7.333333,2.716760,1.256410,2.571429,1.615919,0.795455,3.093750,1.825642
EUW1_6045620377,0.0,5.000000,1.000000,0.0,0.000000,0.000000,1.054054,3.111111,2.045852,1.666667,3.105263,2.157062
...,...,...,...,...,...,...,...,...,...,...,...,...
EUW1_6135052656,0.0,6.200000,2.040000,0.0,2.000000,0.400000,1.419355,6.200000,2.768603,0.547619,1.966667,1.428716
EUW1_6141595905,0.0,3.060606,0.901010,0.0,1.750000,0.350000,2.181818,7.230769,3.402603,1.642857,3.166667,2.256667
EUW1_6137361607,0.0,4.500000,0.900000,0.0,1.375000,0.275000,1.361111,3.000000,2.465799,2.024390,3.666667,3.022082
EUW1_6088300233,0.0,3.500000,0.700000,0.0,0.000000,0.000000,1.392157,3.454545,2.188451,1.555556,2.392857,2.053812


### Conversion des booléens
Calcul d'un "score", qui représente le nombre de joueurs possédant la caractéristique

In [58]:
def add_bool_score_column(datasetInput,datasetOutput, boolvar):
    listCols = ['TOP0_','TOP1_','JGL0_','JGL1_','MID0_','MID1_','ADC0_','ADC1_','SUP0_','SUP1_']
    datasetInterm = datasetInput[[c+boolvar for c in listCols]]
    datasetInterm.applymap(int)
    datasetOutput[boolvar+'_SCORE_0'] = datasetInterm['TOP0_'+boolvar]
    datasetOutput[boolvar+'_SCORE_0'] = 0
    datasetOutput[boolvar+'_SCORE_1'] = 0
    for i in range(len(listCols)):
        if i%2==0:
            datasetOutput[boolvar+'_SCORE_0'] = datasetOutput[boolvar+'_SCORE_0'] + datasetInterm[listCols[i]+boolvar]
        else:
            datasetOutput[boolvar+'_SCORE_1'] = datasetOutput[boolvar+'_SCORE_1'] + datasetInterm[listCols[i]+boolvar]

In [59]:
#Test
df = pd.DataFrame()
add_bool_score_column(dataset,df,'FILL')
add_bool_score_column(dataset,df,'HOT')
df

,FILL_SCORE_0,FILL_SCORE_1,HOT_SCORE_0,HOT_SCORE_1
EUW1_6140228604,2,1,0,1
EUW1_6141546191,2,1,0,0
EUW1_6140416681,5,3,0,0
EUW1_6139592219,1,1,0,0
EUW1_6045620377,5,2,0,0
...,...,...,...,...
EUW1_6135052656,3,3,0,0
EUW1_6141595905,2,1,0,1
EUW1_6137361607,3,3,1,1
EUW1_6088300233,3,3,1,0


### 4. Ajout des données qui doivent être mises en échelle log

### 5. Ajout des données qui n'ont pas besoin d'être transformées

In [54]:
def add_numerical_column(datasetInput,datasetOutput,col):
    min_max_moy(datasetOutput,datasetInput,col,'0')
    min_max_moy(datasetOutput,datasetInput,col,'1')

In [56]:
#Test
df = pd.DataFrame()
add_numerical_column(dataset,df,'GWR')
df

,GWR0_MIN,GWR0_MAX,GWR0_MOY,GWR1_MIN,GWR1_MAX,GWR1_MOY
EUW1_6140228604,0.482474,0.506329,0.494504,0.496700,0.543860,0.526773
EUW1_6141546191,0.407407,0.512031,0.481251,0.494977,0.560185,0.515126
EUW1_6140416681,0.437500,0.500000,0.469306,0.484127,0.540856,0.504985
EUW1_6139592219,0.496063,0.550725,0.523601,0.490775,0.593886,0.544498
EUW1_6045620377,0.479371,0.524487,0.507712,0.471204,0.520000,0.491642
...,...,...,...,...,...,...
EUW1_6135052656,0.483425,0.574468,0.514136,0.502640,0.528926,0.517826
EUW1_6141595905,0.419355,0.520958,0.489419,0.458333,0.557522,0.511513
EUW1_6137361607,0.421053,0.630435,0.523385,0.446154,0.541451,0.497452
EUW1_6088300233,0.487500,0.558376,0.515654,0.468354,0.519084,0.494472
